## Computing optimal partitions of the sphere

The first objective function is  the diameter functional defined on the sphere partitions
The second objective function is  the ratio (max diam)/(min dist), where (max diam) is the maximal diameter of a region, and (min dist) is the minimal distance between regions which are at dual graph distance 3.




In [48]:
# reading the solution of the Tomson problem from the database

num_filename = '20.xyz'
!wget https://www-wales.ch.cam.ac.uk/~wales/CCD/Thomson/xyz/{num_filename}

--2023-07-09 09:50:35--  https://www-wales.ch.cam.ac.uk/~wales/CCD/Thomson/xyz/20.xyz
Resolving www-wales.ch.cam.ac.uk (www-wales.ch.cam.ac.uk)... 193.60.92.99
Connecting to www-wales.ch.cam.ac.uk (www-wales.ch.cam.ac.uk)|193.60.92.99|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 995 [chemical/x-xyz]
Saving to: ‘20.xyz’

20.xyz              100%[===================>]     995  --.-KB/s    in 0s      

2023-07-09 09:50:35 (91.9 MB/s) - ‘20.xyz’ saved [995/995]



In [49]:


import scipy.optimize as opt
import numpy as np
from numpy import pi
import networkx as nx
from math import acos
import matplotlib.pyplot as plt
from numpy.linalg import norm
from numba import jit, njit

import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import sys
from tqdm import tqdm
from scipy.spatial.distance import pdist
#import pandas as pd



def read_points(filename):
    f = open(filename,'r')
    k = 0
    l = 0
    pt = []
    for s in f:
        pt.append(float(s))
        l = (l+1)%3
        if l==0:
            pts.append(pt)
            pt=[]
            l =0
            k+=1
    print(k)
    #....

def read_points2(filename):
    f = open(filename,'r')
    k = int(f.readline())
    f.readline()
    for s in f:
        l = s.split(' ')
        #print(l)
        pts.append([float(l[1]),float(l[2]),float(l[3])])

def angle(p1,p2):
    c = p1[0]*p2[0]+p1[1]*p2[1]+p1[2]*p2[2]
    sq1 = p1[0]*p1[0]+p1[1]*p1[1]+p1[2]*p1[2]
    sq2 = p2[0]*p2[0]+p2[1]*p2[1]+p2[2]*p2[2]
    try:
        return acos(c/(sq1*sq2)**0.5)
    except ValueError:
        #print(c,' ',sq1,' ',sq2)
        return pi

@jit
def middle(x,y):   # middle point of the arc
    z = [(x[0]+y[0])/2, (x[1]+y[1])/2, (x[2]+y[2])/2]
    l = (z[0]*z[0]+z[1]*z[1]+z[2]*z[2])**0.5
    z = [z[0]/l,z[1]/l,z[2]/l]
    return z


def get_graph():   # add edge, if angle_min< a < angle_max
    g = nx.Graph()
    n = len(pts)
    for i in range(n):
        for j in range(i+1,n):
            a = angle(list(pts[i]),list(pts[j]))
            if (a>angle_min) and (a<angle_max):
                g.add_edge(i,j)

    cl =  [c for c in nx.find_cliques(g) if len(c)>3]
    for c in cl:
      ma = 0.0
      me = []
      for i in c:
        for j in c:
          if i>j:
            a = angle(pts[i],pts[j])
            if a> ma:
              ma = a
              me = [i,j]
      if me and me[1] in g[me[0]]:
        g.remove_edge(me[0],me[1])
    return g

def center(x,y,z):    # circumcenter, a vertex of the Voronoi diagram
    x0=[0.0,0.0,0.0]
    x0[0] = (x[0]+y[0]+z[0])/3
    x0[1] = (x[1]+y[1]+z[1])/3
    x0[2] = (x[2]+y[2]+z[2])/3
    l = (x0[0]*x0[0]+x0[1]*x0[1]+x0[2]*x0[2])**0.5
    x0[0]=x0[0]/l
    x0[1]=x0[1]/l
    x0[2]=x0[2]/l
#    print(x0)
    f = lambda v: (v[0]*x[0]+v[1]*x[1]+v[2]*x[2] - v[0]*y[0]-v[1]*y[1]-v[2]*y[2])**2 + (v[0]*x[0]+v[1]*x[1]+v[2]*x[2] - v[0]*z[0]-v[1]*z[1]-v[2]*z[2])**2+ (v[0]*v[0]+v[1]*v[1]+v[2]*v[2]-1.0)**2 #+0.000001*((x0[0]-v[0])**2+(x0[1]-v[1])**2+(x0[2]-v[2])**2)


    sol = opt.minimize(f,x0, method='BFGS', tol=1e-12)
    if f(sol.x)>1e-5:
      print(f(sol.x),x,y,z)
    return [sol.x[0],sol.x[1],sol.x[2]]

def get_dual(g):
    for v in range(g.number_of_nodes()):
        neigh = g.neighbors(v)

        try:
            n_cyc = nx.cycle_basis(g.subgraph(neigh))[0]
            n_cyc.append(n_cyc[0])
        except IndexError:
            print(nx.cycle_basis(g.subgraph(neigh)))
        face = []
        for i in range(len(n_cyc)-1):
            c = center(pts[v],pts[n_cyc[i]],pts[n_cyc[i+1]])
            face.append(c)
#        face1 = face.copy()
#        for i in range(len(face1)-1):
#            face.append(middle(face1[i],face1[i+1]))
#        face.append(middle(face1[0],face1[len(face1)-1]))
        faces.append(face)

def dist(u,v):
    x = [u[0]-v[0],u[1]-v[1],u[2]-v[2]]
    return norm(x)

def get_diam():
    for f in faces:
        d = 0.0
        k = len(f)
        for i in range(k):
            for j in range(i+1,k):
                d1 = dist(f[i],f[j])
                if d1>d:
                    d = d1
        diam.append(d)
    return max(diam)


def plane_equation(x,y,z):
    a1 = x[1] - x[0]
    b1 = y[1] - y[0]
    c1 = z[1] - z[0]
    a2 = x[2] - x[0]
    b2 = y[2] - y[0]
    c2 = z[2] - z[0]
    a = b1 * c2 - b2 * c1
    b = a2 * c1 - a1 * c2
    c = a1 * b2 - b1 * a2
    d = (- a * x[0] - b * y[0] - c * z[0])
    return [a, b, c, d]

def foot(x,y,z,v):
    p = plane_equation(x,y,z)
    n = [p[0],p[1],p[2]]
    l = norm(n)
    n = [n[0]/l,n[1]/l,n[2]/l]
    h = p[0]*v[0]+p[1]*v[1]+p[2]*v[2]+p[3]
    n1 = [n[0]*h/l,n[1]*h/l,n[2]*h/l]
    return [v[0]-n1[0],v[1]-n1[1],v[2]-n1[2]]

def circ_dist(x,y,v):
    z = [0.0,0.0,0.0]
    f = foot(x,y,z,v)

    l = norm(f)
    f = [f[0]/l,f[1]/l,f[2]/l]

    a1 = angle(x,f)
    a2 = angle(f,y)
    a3 = angle(x,y)
    d = min(dist(x,v),dist(y,v))
    if abs(a1+a2-a3)<eps:
        d = min(d,dist(f,v))
    return d


def faces_dist(a,b):  
    f1 = faces[a]
    f2 = faces[b]
    d = 100.0

    for i in range(len(f1)):
        for j in range(len(f2)):
            v = f1[i]
            x = f2[j]
            if j==len(f2)-1:
                y = f2[0]
            else:
                y = f2[j+1]
            d = min(d,circ_dist(x,y,v))
    for i in range(len(f2)):
        for j in range(len(f1)):
            v = f2[i]
            x = f1[j]
            if j==len(f1)-1:
                y = f1[0]
            else:
                y = f1[j+1]
            d = min(d,circ_dist(x,y,v))
    return d

def faces_dist_forb(a,b,fd):   
    f1 = faces[a]
    f2 = faces[b]
    d1 = 100.0
    d2 = -1.0

    for i in range(len(f1)):
        for j in range(len(f2)):
            v = f1[i]
            x = f2[j]
            if j==len(f2)-1:
                y = f2[0]
            else:
                y = f2[j+1]
            dcur = circ_dist(x,y,v)
            d1 = min(d1, dcur)
            d2 = max(d2, dcur)

    for i in range(len(f2)):
        for j in range(len(f1)):
            v = f2[i]
            x = f1[j]
            if j==len(f1)-1:
                y = f1[0]
            else:
                y = f1[j+1]
            dcur = circ_dist(x,y,v)
            d1 = min(d1, dcur)
            d2 = max(d2, dcur)
    return d1<fd<d2

def forb_dist_edges(fd):
  fn = len(faces)
  res = []
  for i in range(fn):
    for j in range(i+1,fn):
      if faces_dist_forb(i,j,fd):
        res.append([i,j])
  return res


def faces_d3_dist(g):  
    n = g.number_of_nodes()
    d = 2.0
    for i in range(n):
        for j in range(i+1,n):
            if nx.shortest_path_length(g,i,j)==3:
                d1 = faces_dist(i,j)
                if d1<d:
                    d = d1
                d3edges.append((i,j))
    return d

dual_vert = []

def get_opt_graphs(g):
  g1 = nx.Graph()
  g2 = nx.Graph()
  d = dict()
  fn = 0
  k = 0
  for f in faces:
    vn = 0
    for v in f:
      dual_vert.append(v)
      d[(fn,vn)] = k
      vn += 1
      k += 1
    for p in range(vn):
      for q in range(p+1,vn):
        g1.add_edge(k-vn+p,k-vn+q)
    fn += 1
  for e in d3edges:
    n1 = len(faces[e[0]])
    n2 = len(faces[e[1]])
    for p in range(n1):
      for q in range(n2):
        g2.add_edge(d[(e[0],p)],d[(e[1],q)])
  return g1,g2

def merge_pts(g1,g2):
  h = nx.Graph()
  m = len(dual_vert)
  for i in range(m):
    for j in range(i+1,m):
      if dist(dual_vert[i],dual_vert[j])<1E-3:
        h.add_edge(i,j)
  print(h.number_of_edges())
  eq = lambda u,v: u in h[v]
  g1a = nx.quotient_graph(g1,eq)
  g2a = nx.quotient_graph(g2,eq)
  return g1a, g2a

dual_vert_merge = []

def get_final_graphs(g1a,g2a):
  new = []
  d = dict()
  k = 0
  for v in g1a.nodes():
    new.append(min(v))
    for u in v:
      d[u] = k
    k += 1
  for i in new:
    dual_vert_merge.append(dual_vert[i])
  g1b = nx.Graph()
  g2b = nx.Graph()
  for e in g1a.edges():
    g1b.add_edge(d[min(e[0])], d[min(e[1])])
    #print(dist(dual_vert_merge[d[min(e[0])]],dual_vert_merge[d[min(e[1])]]))
  for e in g2a.edges():
    g2b.add_edge(d[min(e[0])], d[min(e[1])])
  return g1b, g2b


def set_dist():
  global angle_min
  global angle_max
  n = len(pts)
  all_dist = []
  for i in range(n):
    for j in range(i+1,n):
      all_dist.append(dist(pts[i],pts[j]))
  d0 = min(all_dist)*0.999
  d1 = d0*1.5
  angle_min = 2*np.arcsin(d0/2)
  print("angle_min = ", angle_min)
  angle_max = 2*np.arcsin(d1/2)
  print("angle_max = ", angle_max)





In [50]:
pts = []     # points of the dual triangulation
faces = []
diam = []    # diameters
eps = 1E-8

d3edges = []

read_points2(num_filename)
set_dist()

g = get_graph()

print(g.number_of_edges())

get_dual(g)
d0 = get_diam()
print('max_diam =', d0)
d1 = faces_d3_dist(g)
print('min_dist = ',d1)

print('ratio = ',d1/d0)

g1, g2 = get_opt_graphs(g)
g1a, g2a = merge_pts(g1,g2)
g1b, g2b = get_final_graphs(g1a, g2a)  # graph of *possible* diameters


angle_min =  0.8036293264137523
angle_max =  1.2537916718675781
54
max_diam = 1.032629109839321
min_dist =  1.0468658361345904
ratio =  1.0137868729049142
108


In [51]:
# here some portions of this code has been used: https://github.com/thoppe/tf_thomson_charges

def min_diam_model(vert, diam_graph):
    tf.reset_default_graph()

    coord = tf.Variable(np.array(vert), name='coordinates')

    coord = coord/tf.reshape(tf.norm(coord,axis=1),(-1,1))

    def squared_diff(A):
        r = tf.reduce_sum(A*A, 1)
        r = tf.reshape(r, [-1, 1])
        return r - 2*tf.matmul(A, tf.transpose(A)) + tf.transpose(r)

    RR = squared_diff(coord)

    mask = np.triu(np.array(nx.adjacency_matrix(diam_graph, nodelist=sorted(diam_graph.nodes())).todense(), dtype=np.bool_), 1)
    R = tf.boolean_mask(RR, mask)

    U = tf.reduce_max(R)

    return U, coord

def minimize_diam():

    D2, coord = min_diam_model(dual_vert_merge,g1b)

    previous_d2 = 4.0

    learning_rate = 0.01
    LR = tf.placeholder(tf.float64, shape=[])
    opt = tf.train.AdamOptimizer(learning_rate=LR).minimize(D2)

    with tf.Session() as sess:
        init = tf.global_variables_initializer()
        sess.run(init)

        d2 = sess.run(D2)

        print(" iteration / N / D2_current / delta / learning rate")

        for n in range(100000):
            for _ in range(100):
                sess.run(opt, feed_dict={LR:learning_rate})

            d2 = sess.run(D2)
            delta_d2 = np.abs(previous_d2 - d2)
            previous_d2 = d2

            msg = "  {:0.14f} {:0.14f} {:0.10f}"

            print(msg.format(d2, delta_d2, learning_rate))

            if np.isclose(delta_d2,0,atol=1e-10):
                break

            if d2<1E-3:
                break

            learning_rate *= 0.96

        d2,c = sess.run([D2,coord])
        return np.sqrt(d2), c



##Minimizing the maximal diameter

In [52]:
res, coord = minimize_diam()
res

 iteration / N / D2_current / delta / learning rate
  1.04017518877405 2.95982481122595 0.0100000000
  1.01468281193013 0.02549237684392 0.0096000000
  1.03008478963563 0.01540197770550 0.0092160000
  1.01207338955716 0.01801140007847 0.0088473600
  1.02385883090150 0.01178544134434 0.0084934656
  1.01431869309954 0.00954013780195 0.0081537270
  1.01558124348448 0.00126255038493 0.0078275779
  1.00093441808772 0.01464682539676 0.0075144748
  1.01140173227813 0.01046731419041 0.0072138958
  1.01050207211807 0.00089966016006 0.0069253400
  1.01081258758304 0.00031051546497 0.0066483264
  0.99883408065087 0.01197850693218 0.0063823933
  0.99799809998072 0.00083598067015 0.0061270976
  0.99893076223364 0.00093266225293 0.0058820137
  1.00107399890622 0.00214323667258 0.0056467331
  0.99124613008507 0.00982786882116 0.0054208638
  0.99165940127204 0.00041327118697 0.0052040292
  0.99680544920843 0.00514604793639 0.0049958681
  0.99564318509364 0.00116226411479 0.0047960334
  0.9867348021755

0.9814717796377683

In [53]:
def max_ratio_model(vert, diam_graph, d3_graph):
    tf.reset_default_graph()

    coord = tf.Variable(np.array(vert), name='coordinates')

    coord = coord/tf.reshape(tf.norm(coord,axis=1),(-1,1))

    def squared_diff(A):
        r = tf.reduce_sum(A*A, 1)
        r = tf.reshape(r, [-1, 1])
        return r - 2*tf.matmul(A, tf.transpose(A)) + tf.transpose(r)

    RR = squared_diff(coord)

    mask1 = np.triu(np.array(nx.adjacency_matrix(diam_graph, nodelist=sorted(diam_graph.nodes())).todense(), dtype=np.bool_), 1)
    mask2 = np.triu(np.array(nx.adjacency_matrix(d3_graph, nodelist=sorted(d3_graph.nodes())).todense(), dtype=np.bool_), 1)
    diams = tf.boolean_mask(RR, mask1)
    dmax = tf.reduce_max(diams)

    dists = tf.boolean_mask(RR, mask2)
    dist_min = tf.reduce_min(dists)

    ratio = dist_min/dmax

    return -ratio, coord

def maximize_ratio():

    D2, coord = max_ratio_model(dual_vert_merge,g1b,g2b)

    previous_d2 = 4.0

    learning_rate = 0.01
    LR = tf.placeholder(tf.float64, shape=[])
    opt = tf.train.AdamOptimizer(learning_rate=LR).minimize(D2)

    with tf.Session() as sess:
        init = tf.global_variables_initializer()
        sess.run(init)

        d2 = sess.run(D2)

        print(" iteration / N / D2_current / delta / learning rate")

        for n in range(100000):
            for _ in range(100):
                sess.run(opt, feed_dict={LR:learning_rate})

            d2 = sess.run(D2)
            delta_d2 = np.abs(previous_d2 - d2)
            previous_d2 = d2

            msg = "  {:0.14f} {:0.14f} {:0.10f}"

            print(msg.format(d2, delta_d2, learning_rate))

            if np.isclose(delta_d2,0,atol=1e-10):
                break

            if d2<-1.9999:
                break

            learning_rate *= 0.97

        d2,c = sess.run([D2,coord])
        return np.sqrt(-d2), c


##Maximizing the ratio

In [54]:
rat, coord = maximize_ratio()
rat

 iteration / N / D2_current / delta / learning rate
  -1.28959099222736 5.28959099222736 0.0100000000
  -1.32040177711215 0.03081078488479 0.0097000000
  -1.33281896031586 0.01241718320370 0.0094090000
  -1.33410179404148 0.00128283372562 0.0091267300
  -1.34803603936924 0.01393424532777 0.0088529281
  -1.32145225546940 0.02658378389985 0.0085873403
  -1.33349449948082 0.01204224401142 0.0083297200
  -1.33991492071519 0.00642042123437 0.0080798284
  -1.35546355174681 0.01554863103162 0.0078374336
  -1.35495152855512 0.00051202319170 0.0076023106
  -1.33753652301465 0.01741500554046 0.0073742413
  -1.35027959209124 0.01274306907658 0.0071530140
  -1.35165660235367 0.00137701026243 0.0069384236
  -1.35331045455312 0.00165385219945 0.0067302709
  -1.37012884243885 0.01681838788573 0.0065283628
  -1.36239248411128 0.00773635832757 0.0063325119
  -1.36274478802572 0.00035230391444 0.0061425365
  -1.36175944554416 0.00098534248157 0.0059582604
  -1.36180054975860 0.00004110421445 0.005779512

1.1850399750603855